## Introduction.
Hamburg is the second-largest city in Germany and is known to be the greenest city in the Country, rich in recreational areas.\
Moving inside Hamburg to an area closer to my work, I was looking for a place to live in a neighborhood where I could also easily walk my dog and play basketball in the evenings. Searching for a new dwelling I realized that boroughs of the city distinguishes from each other in this aspect.\
In this project I would like to explore neighborhoods in Hamburg for availability of sports grounds and green areas. Such clustering and its visualisation can provide with an information about parts of the city with a lack of places to rest. This information can be used by management of the city to make Hamburg even more better place to live. Moreover, this report is useful for someone who is looking for a place in Hamburg for recreational purposes.


## Description of the data to be used in the analysis.
The data for the analysis will be combined from two data sets. The first one contains geographical coordinates for every postal code in Hamburg. The second one provides us with information about boroughs and neighborhoods corresponding to each postal code.\
Resulting data set will contain boroughs and neighborhoods, postal codes and related longitude and latitude. It will be used to explore the neighborhoods and segment them with the Foursquare API. With the k-means clustering algorithm we will cluster the neighbourhoods in Hamburg and visualise them for observations and analysis.

Before to get the data and start exploring it, several libraries should be installed.

In [1]:
import requests # library to handle requests

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Datasets

The first dataset contains the data of all german postal codes, cities and their coordinataes

In [3]:
url='https://launix.de/launix/wp-content/uploads/2019/06/PLZ.csv'

In [4]:
headers = ["PostalCode","City","Longitude","Latitude"]

Transform the data into a pandas dataframe and choose the only positions related to Hamburg

In [5]:
df = pd.read_csv(url, skiprows=3, names = headers, sep=';')

In [6]:
df_h=df[df['City']=='Hamburg'].reset_index(drop=True)

df_h.head()

,PostalCode,City,Longitude,Latitude
0,20095,Hamburg,10.001104,53.554158
1,20097,Hamburg,10.020163,53.549704
2,20099,Hamburg,10.011777,53.560801
3,20144,Hamburg,9.975633,53.577054
4,20146,Hamburg,9.978171,53.570563


Dowmload and explore the second dataset with Postal Codes, Boroughs and Neighbourhoods of Hamburg.

In [7]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [8]:
#the datasets columns need to be renamed because of german transcription
column_names= ['1','2','3','4','5','6','7','8','Neighborhood', 'Borough',"PostalCode",'12']
url1='https://www.statistik-nord.de/fileadmin/Dokumente/Wahlen/Hamburg/Bezirksversammlungswahlen/2019/Vor_der_Wahl/2018-06-19Stra%C3%9FenHH_WahlkreisBVWahl_Stadtteil_PLZ.xlsx'

In [9]:
df_plz = pd.read_excel(url1,names = column_names)

In [10]:
#drop the first row because it's empty
df_plz = df_plz.iloc[1:]

In [11]:
df_plz.head(3)

,1,2,3,4,5,6,7,8,Neighborhood,Borough,PostalCode,12
1,Aalheitengraben,1,NaN,3,NaN,1,57,"Lemsahl-Mellingstedt, Duvenstedt, Wohldorf-Ohl...",Volksdorf,Wandsbek,22359,A001
2,Aalheitengraben,4,NaN,14,NaN,2,57,"Lemsahl-Mellingstedt, Duvenstedt, Wohldorf-Ohl...",Volksdorf,Wandsbek,22359,A001
3,Aalkrautweg,1,NaN,35,a,1,55,"Wellingsbüttel, Sasel",Sasel,Wandsbek,22395,A002


In [12]:
#Let's drop all usless for the analysis columns 
df_test=df_plz.drop(['1','2','3','4','5','6','7','8','12'], axis=1)
df_test.head()

,Neighborhood,Borough,PostalCode
1,Volksdorf,Wandsbek,22359
2,Volksdorf,Wandsbek,22359
3,Sasel,Wandsbek,22395
4,Sasel,Wandsbek,22395
5,Sasel,Wandsbek,22393


at this step we will merge both datasets on postal codes 

In [13]:
df_hamburg = pd.merge(df_h, df_test, on='PostalCode',how='inner')
df_hamburg.head()

,PostalCode,City,Longitude,Latitude,Neighborhood,Borough
0,20095,Hamburg,10.001104,53.554158,Hamburg-Altstadt,Hamburg-Mitte
1,20095,Hamburg,10.001104,53.554158,Hamburg-Altstadt,Hamburg-Mitte
2,20095,Hamburg,10.001104,53.554158,Hammerbrook,Hamburg-Mitte
3,20095,Hamburg,10.001104,53.554158,Hamburg-Altstadt,Hamburg-Mitte
4,20095,Hamburg,10.001104,53.554158,Hamburg-Altstadt,Hamburg-Mitte


In [14]:
# let's check the shape
df_hamburg.shape

(15491, 6)

In [15]:
#drop duplicates
df_hamburg.sort_values("Neighborhood", inplace = True)
df_hamburg.drop_duplicates(subset="Neighborhood", keep='first', inplace=True)

In [16]:
df_hamburg.reset_index(drop=True)

,PostalCode,City,Longitude,Latitude,Neighborhood,Borough
0,21037,Hamburg,10.144101,53.450581,Allermöhe,Bergedorf
1,22297,Hamburg,10.012196,53.611496,Alsterdorf,Hamburg-Nord
2,21039,Hamburg,10.261889,53.455322,Altengamme,Bergedorf
3,21129,Hamburg,9.848822,53.524423,Altenwerder,Harburg
4,22767,Hamburg,9.947594,53.553175,Altona-Altstadt,Altona
5,22765,Hamburg,9.932502,53.557255,Altona-Nord,Altona
6,22607,Hamburg,9.879987,53.571865,Bahrenfeld,Altona
7,22305,Hamburg,10.049692,53.591714,Barmbek-Nord,Hamburg-Nord
8,22085,Hamburg,10.015176,53.574655,Barmbek-Süd,Hamburg-Nord
9,21035,Hamburg,10.150629,53.494515,Bergedorf,Bergedorf


The Dataset is ready. Let's check its size

In [17]:
print('the number of uniq neighborhoods is')
df_hamburg.shape

the number of uniq neighborhoods is


(103, 6)

Now we will define Foursquare Credentials and Version

In [18]:
CLIENT_ID = 'PRGJP2KTHQJW2GSA1O4BJC1UTQCBLQ5SPTGOAH4BN3TXAUAF' # your Foursquare ID
CLIENT_SECRET = 'A4V0FBT01TBMDP4E24R52W2K3SYQNZF023AT1ZGV0D2MCZ3X' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PRGJP2KTHQJW2GSA1O4BJC1UTQCBLQ5SPTGOAH4BN3TXAUAF
CLIENT_SECRET:A4V0FBT01TBMDP4E24R52W2K3SYQNZF023AT1ZGV0D2MCZ3X


In [49]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [50]:
import folium # map rendering library

In [51]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done

# All requested packages already installed.



In [52]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [53]:
address = 'Hamburg'

geolocator = Nominatim(user_agent="h_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hamburg are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hamburg are 53.550341, 10.000654.


let's visualize the neighborhoods in Hamburg.

In [54]:
# create map of Hamburg using latitude and longitude values
map_hamburg = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_hamburg['Latitude'], df_hamburg['Longitude'], df_hamburg['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hamburg)  
    
map_hamburg

In [19]:
# let's explore the first neighborhood in our dataframe
neighborhood_latitude = df_hamburg.loc[0,'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_hamburg.loc[0,'Longitude'] # neighborhood longitude value

neighborhood_name = df_hamburg.loc[0,'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hamburg-Altstadt are 53.5541579534295, 10.0011036114924.


Now, let's get the top 100 venues that are within a radius of 500 meters.\
First, let's create the GET request URL.

In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=PRGJP2KTHQJW2GSA1O4BJC1UTQCBLQ5SPTGOAH4BN3TXAUAF&client_secret=A4V0FBT01TBMDP4E24R52W2K3SYQNZF023AT1ZGV0D2MCZ3X&v=20180605&ll=53.5541579534295,10.0011036114924&radius=500&limit=100'

Extract and Explore Neighborhood by Venue

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '609aacb5d2c21539de9c6439'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Hamburg-Altstadt',
  'headerFullLocation': 'Hamburg-Altstadt, Hamburg',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 102,
  'suggestedBounds': {'ne': {'lat': 53.558657957929505,
    'lng': 10.008664433891797},
   'sw': {'lat': 53.5496579489295, 'lng': 9.993542789093004}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b0d3ac1f964a520dd4423e3',
       'name': 'Sakura Sushi',
       'location': {'address': 'Rosenstr. 8',
        'lat': 53.552441853233546,
        'lng': 10.000058291854657,
        'labeledLatLngs': [{'label': 'display',
          'lat

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Sakura Sushi,Sushi Restaurant,53.552442,10.000058
1,Hamburger Kunsthalle - Galerie der Gegenwart,Art Museum,53.555376,10.002739
2,Barceló Hamburg,Hotel,53.554432,10.000625
3,Quan Do,Vietnamese Restaurant,53.553373,10.002558
4,Thalia Theater,Theater,53.552000,9.998338


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's run a code to apply the above function to each neighborhood 

In [25]:
hamburg_venues = getNearbyVenues(names=df_hamburg['Neighborhood'],
                                   latitudes=df_hamburg['Latitude'],
                                   longitudes=df_hamburg['Longitude']
                                  )

Allermöhe
Alsterdorf
Altengamme
Altenwerder
Altona-Altstadt
Altona-Nord
Bahrenfeld
Barmbek-Nord
Barmbek-Süd
Bergedorf
Bergstedt
Billbrook
Billstedt
Billwerder
Blankenese
Borgfelde
Bramfeld
Cranz
Curslack
Dulsberg
Duvenstedt
Eidelstedt
Eilbek
Eimsbüttel
Eißendorf
Eppendorf
Farmsen-Berne
Finkenwerder
Francop
Fuhlsbüttel
Groß Borstel
Groß Flottbek
Gut Moor
HafenCity
Hamburg-Altstadt
Hamm
Hammerbrook
Harburg
Harvestehude
Hausbruch
Heimfeld
Hoheluft-Ost
Hoheluft-West
Hohenfelde
Horn
Hummelsbüttel
Iserbrook
Jenfeld
Kirchwerder
Kleiner Grasbrook
Langenbek
Langenhorn
Lemsahl-Mellingstedt
Lohbrügge
Lokstedt
Lurup
Marienthal
Marmstorf
Moorburg
Neuallermöhe
Neuenfelde
Neuengamme
Neugraben-Fischbek
Neuland
Neustadt
Neuwerk
Niendorf
Nienstedten
Ochsenwerder
Ohlsdorf
Osdorf
Othmarschen
Ottensen
Poppenbüttel
Rahlstedt
Reitbrook
Rissen
Rothenburgsort
Rotherbaum
Rönneburg
Sasel
Schnelsen
Sinstorf
Spadenland
St. Georg
St. Pauli
Steilshoop
Steinwerder
Stellingen
Sternschanze
Sülldorf
Tatenberg
Tonndorf
U

In [26]:
hamburg_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allermöhe,53.450581,10.144101,SC4M,53.449580,10.135803,Soccer Field
1,Allermöhe,53.450581,10.144101,Sporthalle Fünfhausen,53.449196,10.135578,Theme Park
2,Allermöhe,53.450581,10.144101,Braatz Informatik,53.445650,10.134290,IT Services
3,Allermöhe,53.450581,10.144101,Paddel Meier,53.458001,10.139787,Harbor / Marina
4,Alsterdorf,53.611496,10.012196,Eiskaffee Eis Perle,53.608354,10.009394,Ice Cream Shop


Let's check how many venues were returned for each neighborhood

In [27]:
hamburg_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allermöhe,4,4,4,4,4,4
Alsterdorf,29,29,29,29,29,29
Altenwerder,2,2,2,2,2,2
Altona-Altstadt,100,100,100,100,100,100
Altona-Nord,100,100,100,100,100,100
Bahrenfeld,15,15,15,15,15,15
Barmbek-Nord,65,65,65,65,65,65
Barmbek-Süd,100,100,100,100,100,100
Bergedorf,6,6,6,6,6,6


Let's find out how many unique categories can be curated from all the returned venues

In [28]:
print('There are {} uniques categories.'.format(len(hamburg_venues['Venue Category'].unique())))

There are 279 uniques categories.


# Explore the venues that contais Parks or Sport in its category

In [29]:
hamburg_venues[hamburg_venues['Venue Category'].str.contains("Park|Sport")]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Allermöhe,53.450581,10.144101,Sporthalle Fünfhausen,53.449196,10.135578,Theme Park
22,Alsterdorf,53.611496,10.012196,Alsterufer,53.613294,10.002261,Park
37,Altona-Altstadt,53.553175,9.947594,Festland,53.554664,9.951444,Water Park
44,Altona-Altstadt,53.553175,9.947594,Wohlers Park (Friedhof Norderreihe),53.558589,9.953012,Park
67,Altona-Altstadt,53.553175,9.947594,Altonaer Turnverband (ATV),53.548921,9.949059,Sports Club
191,Altona-Nord,53.557255,9.932502,Kemal-Altun-Platz,53.552490,9.925879,Park
308,Barmbek-Nord,53.591714,10.049692,Wendebecken,53.598710,10.054479,Park
334,Barmbek-Süd,53.574655,10.015176,Alsterpark,53.568867,10.013555,Park
343,Barmbek-Süd,53.574655,10.015176,Alstervorland,53.573779,10.001142,Park
367,Barmbek-Süd,53.574655,10.015176,Parkbänke Bellevue,53.577069,10.004595,Park


Let's assign this data set to a new dataframe

In [38]:
df_h=hamburg_venues[hamburg_venues['Venue Category'].str.contains("Park|Sport")]
df_h.shape

(93, 7)

there were found 93 locations that meet the requariment

In [55]:
# create map of Hamburg using latitude and longitude values
map_hamburg1 = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(hamburg_venues1['Venue Latitude'], hamburg_venues1['Venue Longitude'], hamburg_venues1['Venue Category']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue' ,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hamburg1)  
    
map_hamburg1

From this map we can can conclude that the North part of Hamburg contains the most Park and Sport Venues, whereas the South part has almost no such locations. In reality it is not so. South Hamburg is full of green and sport areas as well. It seems that Foresquare database contains incomplete information about Hamburg and schould be extended with data for this region to provide us with sufitiant information for further analysis